In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import pylab as pl
from sklearn.metrics import confusion_matrix,accuracy_score

In [2]:
train_path="dataset"
class_names=os.listdir(train_path)

In [5]:
print(class_names)

0

In [ ]:
image_paths=[]
image_classes=[]

In [ ]:
def img_list(path):
    return (os.path.join(path,f) for f in os.listdir(path))

In [6]:
for training_name in class_names:
    dir_=os.path.join(train_path,training_name)
    class_path=img_list(dir_)
    image_paths+=class_path

In [ ]:
len(image_paths)

In [ ]:
image_classes_0=[0]*(len(image_paths)//2)

In [58]:
image_classes_1=[1]*(len(image_paths)//2)

In [59]:
image_classes=image_classes_0+image_classes_1

In [60]:
D=[]

In [ ]:
for i in range(len(image_paths)):
    D.append((image_paths[i],image_classes[i]))

In [ ]:
dataset = D
random.shuffle(dataset)
train = dataset[:180]
test = dataset[180:]

image_paths, y_train = zip(*train)
image_paths_test, y_test = zip(*test)

In [ ]:
des_list=[]

In [ ]:
orb=cv2.ORB_create()

In [ ]:
im=cv2.imread(image_paths[1])

In [ ]:
plt.imshow(im)

In [ ]:
def draw_keypoints(vis, keypoints, color = (0, 255, 255)):
    for kp in keypoints:
            x, y = kp.pt
            plt.imshow(cv2.circle(vis, (int(x), int(y)), 2, color))

In [ ]:
kp = orb.detect(im,None)
kp, des = orb.compute(im, kp)
img=draw_keypoints(im,kp)

In [ ]:
for image_pat in image_paths:
    im=cv2.imread(image_pat)
    kp=orb.detect(im,None)
    keypoints,descriptor= orb.compute(im, kp)
    des_list.append((image_pat,descriptor))

In [ ]:
descriptors=des_list[0][1]
for image_path,descriptor in des_list[1:]:
    descriptors=np.vstack((descriptors,descriptor))

In [ ]:
descriptors.shape

In [ ]:
descriptors_float=descriptors.astype(float)

In [ ]:
from scipy.cluster.vq import kmeans,vq

In [ ]:
k=200
voc,variance=kmeans(descriptors_float,k,1)

In [ ]:
im_features=np.zeros((len(image_paths),k),"float32")
for i in range(len(image_paths)):
    words,distance=vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w]+=1

In [ ]:
from sklearn.preprocessing import StandardScaler
stdslr=StandardScaler().fit(im_features)
im_features=stdslr.transform(im_features)

In [ ]:
from sklearn.svm import LinearSVC
clf=LinearSVC(max_iter=80000)
clf.fit(im_features,np.array(y_train))

In [ ]:
des_list_test=[]

In [ ]:
for image_pat in image_paths_test:
    image=cv2.imread(image_pat)
    kp=orb.detect(image,None)
    keypoints_test,descriptor_test= orb.compute(image, kp)
    des_list_test.append((image_pat,descriptor_test))

In [ ]:
len(image_paths_test)

In [ ]:
from scipy.cluster.vq import vq
test_features=np.zeros((len(image_paths_test),k),"float32")
for i in range(len(image_paths_test)):
    words,distance=vq(des_list_test[i][1],voc)
    for w in words:
        test_features[i][w]+=1

In [ ]:
test_features

In [ ]:
test_features=stdslr.transform(test_features)

In [ ]:
true_classes=[]
for i in y_test:
    if i==1:
        true_classes.append("Cat")
    else:
        true_classes.append("Dog")

In [ ]:
predict_classes=[]
for i in clf.predict(test_features):
    if i==1:
        predict_classes.append("Cat")
    else:
        predict_classes.append("Dog")

In [ ]:
clf.predict(test_features)

In [ ]:
accuracy=accuracy_score(true_classes,predict_classes)
print(accuracy)